<a href="https://colab.research.google.com/github/0xVolt/whats-up-doc/blob/main/src/experimental-notebooks/compare-models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Compare Various HuggingFace Models for CDG by `SmoothedBLEU` Score

## Purpose

The purpose of this notebook is to figure out which models will work best for our CLI app. We will calculate and compare these select models by their smoothed BLEU scores and possibly look into fine-tuning them with [this dataset](https://www.dropbox.com/sh/488bq2of10r4wvw/AACs5CGIQuwtsD7j_Ls_JAORa/finetuning_dataset?dl=0&subfolder_nav_tracking=1).

## Models

These are the models we'll create inference points for and load in this notebook.
1. [SEBIS/code_trans_t5_base_code_documentation_generation_python](https://huggingface.co/SEBIS/code_trans_t5_base_code_documentation_generation_python)
2. [Salesforce/codet5-base-multi-sum](https://huggingface.co/Salesforce/codet5-base-multi-sum)
3. [google/flan-t5-small](https://huggingface.co/google/flan-t5-small)

The models listed above have been fine-tuned on the CodeSearchNet dataset across various programming languages for PL-NL sequence-to-sequence tasks.

## Install Dependencies

In [1]:
%pip install -q transformers sentencepiece datasets nltk accelerate

Note: you may need to restart the kernel to use updated packages.


In [2]:
from transformers import AutoTokenizer, AutoModelWithLMHead, RobertaTokenizer, T5Tokenizer, T5ForConditionalGeneration, SummarizationPipeline
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from datasets import load_dataset
import tokenize
import io

c:\Users\deshi\anaconda3\envs\py39-torch\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Define Evaluation Metric

In [3]:
def calculateSmoothedBLEU(reference, candidate):
    """
    Calculate the Smoothed BLEU-4 score between a reference string and a candidate string.

    Args:
    reference (str): The reference (ground truth) string.
    candidate (str): The candidate (generated) string.

    Returns:
    float: The Smoothed BLEU-4 score.
    """
    reference_tokens = reference.split()
    candidate_tokens = candidate.split()

    smoother = SmoothingFunction()
    bleu_score = sentence_bleu([reference_tokens], candidate_tokens, smoothing_function=smoother.method1)

    return bleu_score

## Define Python Tokenizer

In [4]:
def pythonTokenizer(line):
    result = []
    line = io.StringIO(line)

    for tokenType, token, start, end, line in tokenize.generate_tokens(line.readline):
        if (not tokenType == tokenize.COMMENT):
            if tokenType == tokenize.STRING:
                result.append("CODE_STRING")
            elif tokenType == tokenize.NUMBER:
                result.append("CODE_INTEGER")
            elif (not token == "\n") and (not token == "    "):
                result.append(str(token))
                
    return ' '.join(result)

## Define Models

### CodeTransT5

In [5]:
codeTransModel = AutoModelWithLMHead.from_pretrained("SEBIS/code_trans_t5_base_source_code_summarization_python_transfer_learning_finetune")
codeTransTokenizer = AutoTokenizer.from_pretrained("SEBIS/code_trans_t5_base_source_code_summarization_python_transfer_learning_finetune", skip_special_tokens=True)

c:\Users\deshi\anaconda3\envs\py39-torch\lib\site-packages\transformers\models\auto\modeling_auto.py:1509: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


### CodeT5

In [6]:
codeT5Model = T5ForConditionalGeneration.from_pretrained('Salesforce/codet5-base-multi-sum')
codeT5Tokenizer = RobertaTokenizer.from_pretrained('Salesforce/codet5-base-multi-sum', skip_special_tokens=True)

### FlanT5 

In [7]:
flanT5Model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-small", device_map="auto")
flanT5Tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-small", skip_special_tokens=True)

c:\Users\deshi\anaconda3\envs\py39-torch\lib\site-packages\huggingface_hub\file_download.py:137: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\deshi\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## Create Pipelines

In [8]:
codeTransPipeline = SummarizationPipeline(
    model=codeTransModel,
    tokenizer=codeTransTokenizer,
    device=0
)

In [9]:
codeT5Pipeline = SummarizationPipeline(
    model=codeT5Model,
    tokenizer=codeT5Tokenizer,
    device=0
)

In [11]:
flanPipeline = SummarizationPipeline(
    model=flanT5Model,
    tokenizer=flanT5Tokenizer,
)

## Sample Code

In [12]:
code = '''

def is_prime(number):
    if number <= 1:
        return False
    elif number <= 3:
        return True
    elif number % 2 == 0 or number % 3 == 0:
        return False
    i = 5
    while i * i <= number:
        if number % i == 0 or number % (i + 2) == 0:
            return False
        i += 6
    return True
        
''' #@param {type:"raw"}

tokenizedCode = pythonTokenizer(code)
label = "A code block in Python to check whether a number is prime or not."

## Prompt Models

In [22]:
codeTransOutput = codeTransPipeline([tokenizedCode])[0]['summary_text']
codeT5Output = codeT5Pipeline([tokenizedCode])[0]['summary_text']
flanOutput = flanPipeline([tokenizedCode])[0]['summary_text']

c:\Users\deshi\anaconda3\envs\py39-torch\lib\site-packages\transformers\generation\utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


## Create Dictionary of Predictions

In [23]:
calculateSmoothedBLEU(codeTransOutput, label)

0.04535390934416274

In [24]:
predictions = {
    'codeTrans': {
        'output': codeTransOutput,
        'score': calculateSmoothedBLEU(codeTransOutput, label)
    },
    'codeT5': {
        'output': codeT5Output,
        'score': calculateSmoothedBLEU(codeT5Output, label)
    },
    'flanT5': {
        'output': flanOutput,
        'score': calculateSmoothedBLEU(flanOutput, label)
    }
}

In [25]:
predictions

{'codeTrans': {'output': "What 's the most efficient way to check if an integer is prime in Python ?",
  'score': 0.04535390934416274},
 'codeT5': {'output': 'Check if number is prime .',
  'score': 0.07069301148938888},
 'flanT5': {'output': 'True', 'score': 0}}

---